In [2]:
import sys
import os
savedir = ""
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    savedir = "/content/gdrive/My Drive/NUS/"
    try:
        os.mkdir(savedir)
    except FileExistsError:
        print("Directory \"%s\" already exists" % (savedir))

print("Content of {}:".format(savedir))
os.listdir(savedir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Directory "/content/gdrive/My Drive/NUS/" already exists
Content of /content/gdrive/My Drive/NUS/:


['new.csv', 'new_test.csv']

In [17]:
import numpy as np
import pandas as pd
import lightgbm as lgb

import pandas as pd

### 载入数据
print('载入数据')
trains = pd.read_csv(savedir + 'new.csv')
online_test = pd.read_csv(savedir + 'new_test.csv')

载入数据


In [18]:
### 数据拆分(训练集+验证集+测试集)
print('数据拆分')
from sklearn.model_selection import train_test_split
train_xy,offline_test = train_test_split(trains,test_size = 0.2,random_state=21)
train,val = train_test_split(train_xy,test_size = 0.2,random_state=21)

数据拆分


In [0]:
# 训练集
y_train = train.log_trip_duration                                              # 训练集标签
X_train = train.drop(['id','log_trip_duration'],axis=1)                # 训练集特征矩阵

# 验证集
y_val = val.log_trip_duration                                                  # 验证集标签
X_val = val.drop(['id','log_trip_duration'],axis=1)                    # 验证集特征矩阵

# 测试集
offline_test_X = offline_test.drop(['id','log_trip_duration'],axis=1)  # 线下测试特征矩阵
online_test_X  = online_test.drop(['id','log_trip_duration'],axis=1)              # 线上测试特征矩阵

In [20]:
### 数据转换
print('数据转换')
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

数据转换


In [72]:
### 设置初始参数--不含交叉验证参数
print('设置参数')
params = {
    'boosting_type': 'gbdt', 
    'objective': 'regression',
    'metric': 'rmse',

    'learning_rate': 0.02, 
    'num_leaves': 100, 
    'max_depth': 7,

    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
}

设置参数


In [73]:
cv_results = lgb.cv(
    params, lgb_train, num_boost_round=1000, nfold=5, stratified=False, shuffle=True, metrics='rmse',
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'][-1])

[50]	cv_agg's rmse: 0.472954 + 0.0170533
[100]	cv_agg's rmse: 0.412362 + 0.0170809
[150]	cv_agg's rmse: 0.396853 + 0.0169073
[200]	cv_agg's rmse: 0.389313 + 0.0173887
[250]	cv_agg's rmse: 0.384357 + 0.0178599
[300]	cv_agg's rmse: 0.381484 + 0.0180108
[350]	cv_agg's rmse: 0.379527 + 0.0180528
[400]	cv_agg's rmse: 0.378225 + 0.0181416
[450]	cv_agg's rmse: 0.377235 + 0.0180944
[500]	cv_agg's rmse: 0.376481 + 0.0181318
[550]	cv_agg's rmse: 0.376109 + 0.0181057
[600]	cv_agg's rmse: 0.375719 + 0.0182593
[650]	cv_agg's rmse: 0.375559 + 0.0182333
[700]	cv_agg's rmse: 0.375554 + 0.0181794
[750]	cv_agg's rmse: 0.375382 + 0.0182894
[800]	cv_agg's rmse: 0.375198 + 0.0182893
[850]	cv_agg's rmse: 0.375161 + 0.0183913
best n_estimators: 829
best cv score: 0.37508246631988346


In [77]:
from sklearn.model_selection import GridSearchCV
### 我们可以创建lgb的sklearn模型，使用上面选择的(学习率，评估器数目)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=100,
                              learning_rate=0.02, n_estimators=829, max_depth=7,
                              metric='rmse', bagging_fraction = 0.8,feature_fraction = 0.8)

params_test1={
    'max_depth': range(3,10,2),
    'num_leaves':range(50, 170, 30)
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch1.fit(X_train, y_train)
gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   35.5s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:  1.5min finished


({'max_depth': 7, 'num_leaves': 50}, -0.14123145257188166)

In [79]:
params_test2={
    'max_depth': [6,7,8],
    'num_leaves':[49,50,51]
}

gsearch2 = GridSearchCV(estimator=model_lgb, param_grid=params_test2, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch2.fit(X_train, y_train)
gsearch2.best_params_, gsearch2.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   58.9s finished


({'max_depth': 7, 'num_leaves': 49}, -0.1410988449334604)

In [80]:
params_test3={
    'min_child_samples': [20,22,25,30],
    'min_child_weight':[0.001, 0.002]
}
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=49,
                              learning_rate=0.02, n_estimators=829, max_depth=7, 
                              metric='rmse', bagging_fraction = 0.8, feature_fraction = 0.8)
gsearch3 = GridSearchCV(estimator=model_lgb, param_grid=params_test3, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch3.fit(X_train, y_train)
gsearch3.best_params_, gsearch3.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  1.2min finished


({'min_child_samples': 16, 'min_child_weight': 0.001}, -0.1406210449549232)

In [81]:
params_test4={
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
}
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=49,
                              learning_rate=0.02, n_estimators=829, max_depth=7, 
                              metric='rmse', bagging_freq = 5,  min_child_samples=16)
gsearch4 = GridSearchCV(estimator=model_lgb, param_grid=params_test4, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch4.fit(X_train, y_train)
gsearch4.best_params_, gsearch4.best_score_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   56.9s
[Parallel(n_jobs=4)]: Done 125 out of 125 | elapsed:  2.9min finished


({'bagging_fraction': 1.0, 'feature_fraction': 0.5}, -0.14009529168895737)

In [82]:
params_test6={
    'reg_alpha': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],
    'reg_lambda': [0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5]
}
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=49,
                              learning_rate=0.02, n_estimators=829, max_depth=7, 
                              metric='rmse',  min_child_samples=16, feature_fraction=0.5, bagging_fraction=1.0)
gsearch6 = GridSearchCV(estimator=model_lgb, param_grid=params_test6, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch6.fit(X_train, y_train)
gsearch6.best_params_, gsearch6.best_score_

Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   50.6s


KeyboardInterrupt: ignored

In [83]:
params = {
    'metric' : 'rmse',
    'n_estimators': 829, 
    'feature_fraction': 0.5,
    'bagging_fraction': 1,
    'boosting_type': 'gbdt', 
    'objective': 'regression', 
    'learning_rate': 0.02, 
    'num_leaves': 49, 
    'max_depth': 7,
    'reg_alpha': 0.03, 
    'reg_lambda': 0.001,
    'max_bin': 1000
    }

data_train = lgb.Dataset(X_train, y_train, silent=True)
cv_results = lgb.cv(
    params, data_train, num_boost_round=10000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=100, show_stdv=True)

print('best n_estimators:', len(cv_results['rmse-mean']))
print('best cv score:', cv_results['rmse-mean'][-1])

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's rmse: 0.41895 + 0.0171878
[200]	cv_agg's rmse: 0.39128 + 0.0174324
[300]	cv_agg's rmse: 0.382031 + 0.0175979
[400]	cv_agg's rmse: 0.378419 + 0.0175678
[500]	cv_agg's rmse: 0.377096 + 0.0175487
[600]	cv_agg's rmse: 0.376402 + 0.017572
[700]	cv_agg's rmse: 0.376212 + 0.0176283
[800]	cv_agg's rmse: 0.376124 + 0.0176052
best n_estimators: 791
best cv score: 0.3760514432912583


In [84]:
### 线下预测
params_updated = {
    'boosting_type': 'gbdt', 
    'objective': 'regression', 

    'learning_rate': 0.02, 
    'num_leaves': 49, 
    'max_depth': 7,
    'min_data_in_leaf': 16,
    'n_estimators':791,
    'n_estimators': 829, 
    'feature_fraction': 0.5,
    'bagging_fraction': 1, 
    'max_bin': 1000
}

print ("线下预测")
lgb_model = lgb.train(params_updated, lgb_train, num_boost_round=1500)

from sklearn.metrics import mean_squared_error as MSE
#Make predictions on test data frame
predictions = lgb_model.predict(offline_test_X)
print(np.sqrt(MSE(offline_test.log_trip_duration, predictions)))

线下预测


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.36432597931214405


In [86]:
### 线上预测
print("线上预测")
preds_online = lgb_model.predict(online_test_X)  # 输出概率
print(np.sqrt(MSE(online_test.log_trip_duration,preds_online)))

线上预测
0.3587630711819378


In [0]:
### 特征选择
df = pd.DataFrame(X_train.columns.tolist(), columns=['feature'])
df['importance']=list(lgb_model.feature_importance())                           # 特征分数
df = df.sort_values(by='importance',ascending=False)                      # 特征排序

In [89]:
df

,feature,importance
1,rate_of_lane,2539
0,distance_haversine,2308
7,total_distance,2104
8,total_travel_time,2088
10,direction,2006
2,pressurem,1747
5,tempm,1574
4,dewptm,1440
3,hum,1435
9,number_of_steps,902


In [93]:
import joblib
joblib.dump(lgb_model, savedir + '''train_model.m''')

['/content/gdrive/My Drive/NUS/train_model.m']